In [5]:
import time
import tracemalloc
from collections import deque
import heapq

# ---- Helper Functions ----
def read_input(filepath):
    with open(filepath, 'r') as f:
        lines = [line.strip() for line in f if line.strip()]
    init_idx = lines.index("INITIAL:") + 1
    goal_idx = lines.index("GOAL:") + 1

    initial_rows = lines[init_idx:init_idx+3]
    goal_rows = lines[goal_idx:goal_idx+3]

    initial = []
    for row in initial_rows:
        initial.extend([' ' if c=='_' else c for c in row.split()])

    goal = []
    for row in goal_rows:
        goal.extend([' ' if c=='_' else c for c in row.split()])

    return tuple(initial), tuple(goal)

def write_output(filepath, algo, params, path, t, mem):
    with open(filepath, 'w') as f:
        f.write(f"Algorithm: {algo}\nParams: {params}\nTime(s): {round(t,5)}\nMemory(KB): {round(mem,2)}\n")
        if path:
            f.write(f"Path Length: {len(path)-1}\n\n--- Solution Path ---\n")
            for idx, state in enumerate(path):
                f.write(f"State {idx}:\n")
                for i in range(0, 9, 3):
                    row = [c if c != ' ' else '_' for c in state[i:i+3]]
                    f.write(' '.join(row) + '\n')
                f.write("-----\n")
        else:
            f.write("No solution found.\n")

def check_winner(board):
    wins = [(0,1,2),(3,4,5),(6,7,8),(0,3,6),(1,4,7),(2,5,8),(0,4,8),(2,4,6)]
    for a,b,c in wins:
        if board[a] != ' ' and board[a]==board[b]==board[c]:
            return board[a]
    return None

def is_draw(board):
    return ' ' not in board

def validate_goal(initial, goal):
    x_goal = goal.count('X')
    o_goal = goal.count('O')
    if abs(x_goal - o_goal) > 1:
        return False, None
    if x_goal >= o_goal:
        return True, 'X'
    else:
        return True, 'O'

def next_player(board, start_player='X'):
    x_count = board.count('X')
    o_count = board.count('O')
    if start_player == 'X':
        return 'X' if x_count == o_count else 'O'
    else:
        return 'O' if x_count == o_count else 'X'

def generate_children(board, goal, start_player):
    player = next_player(board, start_player)
    children = []
    for i in range(9):
        if board[i] == ' ':
            new_board = tuple(board[:i]+(player,)+board[i+1:])
            if new_board != goal:
                if check_winner(new_board) or is_draw(new_board):
                    continue
            children.append(new_board)
    return children


# ---- Algorithms ----
def bfs(start, goal, start_player):
    q = deque([(start, [start])])
    visited = {start}
    while q:
        node, path = q.popleft()
        if node == goal:
            return path
        for child in generate_children(node, goal, start_player):
            if child not in visited:
                visited.add(child)
                q.append((child, path+[child]))
    return None

def dfs(start, goal, start_player):
    stack = [(start, [start])]
    visited = {start}
    while stack:
        node, path = stack.pop()
        if node == goal:
            return path
        for child in generate_children(node, goal, start_player):
            if child not in visited:
                visited.add(child)
                stack.append((child, path+[child]))
    return None

def dls(node, goal, start_player, limit, path, visited):
    if node == goal:
        return path
    if limit == 0:
        return None
    for child in generate_children(node, goal, start_player):
        if child not in visited:
            visited.add(child)
            res = dls(child, goal, start_player, limit-1, path+[child], visited)
            if res:
                return res
    return None

def ids(start, goal, start_player, max_depth=9):
    for depth in range(max_depth+1):
        visited = {start}
        path = dls(start, goal, start_player, depth, [start], visited)
        if path:
            return path
    return None

def ucs(start, goal, start_player):
    pq = [(0, start, [start])]
    visited = set()
    while pq:
        cost, node, path = heapq.heappop(pq)
        if node == goal:
            return path
        if node in visited:
            continue
        visited.add(node)
        for child in generate_children(node, goal, start_player):
            heapq.heappush(pq, (cost+1, child, path+[child]))
    return None

def ils(start, goal, start_player):
    limit = 0
    while True:
        pq = [(0, start, [start])]
        visited = set()
        found = False
        while pq:
            cost, node, path = heapq.heappop(pq)
            if node == goal:
                return path
            if node in visited:
                continue
            visited.add(node)
            if cost < limit:
                for child in generate_children(node, goal, start_player):
                    heapq.heappush(pq, (cost+1, child, path+[child]))
        limit += 1
        if limit > 20:
            break
    return None


# ---- Run All Algorithms ----
input_path = '/content/input.txt'
output_base = '/content/'

start_board, goal_board = read_input(input_path)
valid, start_player = validate_goal(start_board, goal_board)

algos = [
    ("BFS", bfs, {}),
    ("DFS", dfs, {}),
    ("DLS", lambda s,g,p: dls(s,g,p,8,[s],{s}), {"depth":8}),
    ("IDS", lambda s,g,p: ids(s,g,p,9), {"max_depth":9}),
    ("UCS", ucs, {}),
    ("ILS", ils, {})
]

if valid:
    for algo_name, algo_func, params in algos:
        tracemalloc.start()
        t0 = time.perf_counter()
        path = algo_func(start_board, goal_board, start_player)
        t1 = time.perf_counter()
        mem = tracemalloc.get_traced_memory()[1]/1024
        tracemalloc.stop()

        write_output(output_base+algo_name+"_output.txt", algo_name, params, path, t1-t0, mem)
else:
    print("Invalid Goal State!")